# Домашнее задание №1

На семинаре мы проходили метрики, необходимо реализовать некоторые из них

In [1]:
import numpy as np
import pandas as pd

## Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


In [55]:
recommended_list = [12, 13, 15, 98, 63, 39, 58, 95, 69, 200]
bought_list = [98, 205, 13] 
np.isin(recommended_list, bought_list)

prices_recommended = [32, 43, 45, 65, 23, 56, 78, 79, 20, 18]
prices_bought = [65, 98, 43]

flags = np.isin(recommended_list, bought_list)
list(flags).index(True)

1

In [43]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])    
    flags = np.isin(recommended_list, bought_list)    
    return flags.sum() / len(bought_list)


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    prices_recommended = np.array(prices_recommended[:k])
    prices_bought = np.array(prices_bought)
    flags = np.isin(recommended_list, bought_list)  
    
    mr = sum(flags * prices_recommended) / sum(prices_bought)
    
    return mr

In [44]:
recall_at_k(recommended_list, bought_list, k=3)

0.3333333333333333

In [45]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=3)

0.2087378640776699

## Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекомендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  MMR(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [58]:
def mrr_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    flags = np.isin(recommended_list, bought_list) 
    rank = list(flags).index(True) + 1
    
    return 1 / rank

In [59]:
mrr_at_k(recommended_list, bought_list, k=5)

0.5

## Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [100]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    flags = np.isin(recommended_list, bought_list) 
    DCG = sum(sum(list(map(lambda x: 1 / np.log2(x + 2), np.where(flags == True))))) #надо плюс два потому что np.where возвращает индекс, а не номер
    iDCG = sum(list(map(lambda x: 1 / np.log2(x + 2), list(range(len(bought_list)))))) #надо плюс два потому что range с нуля нумерует
    
    return DCG / iDCG
    

In [101]:
ndcg_at_k(recommended_list, bought_list, k=5)

0.49818925746641285